In [1]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [2]:
import numpy as np
import heapq
import importlib

import snapshot_obj
import subhalo
import trace_halo
import dataset_compute

In [3]:
importlib.reload(snapshot_obj)
importlib.reload(subhalo)
importlib.reload(trace_halo)
importlib.reload(dataset_compute)

<module 'dataset_compute' from '/home/kasper/Curvaton_Simulations/APOSTLE_analysis/dataset_compute.py'>

In [4]:
%load_ext line_profiler

# Tracing

In [5]:
sim_id = "CDM_V1_LR"
snap_ref = snapshot_obj.Snapshot(sim_id, 127)
snap = snapshot_obj.Snapshot(sim_id, 126)

GNs = snap_ref.get_subhalos('GroupNumber')
SGNs = snap_ref.get_subhalos('SubGroupNumber')

In [18]:
%lprun -f trace_halo.trace_all trace_halo.trace_all(snap_ref, stop=101)

Timer unit: 1e-06 s

Total time: 175.648 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/trace_halo.py
Function: trace_all at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def trace_all(snap_init, gns=[], stop=101):
    10                                               """ Traces all subhalos of given galaxies as far back in time as 
    11                                               possible, starting from the given snapshot.
    12                                           
    13                                               Parameters
    14                                               ----------
    15                                               snap_init : Dataset object
    16                                                   Starting point for the tracing.
    17                                               gns : int, optional
    18                                                   Group 

In [7]:
%lprun -f trace_halo.match_all trace_halo.match_all(snap_ref,snap)

100


Timer unit: 1e-06 s

Total time: 4.04476 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/trace_halo.py
Function: match_all at line 49

Line #      Hits         Time  Per Hit   % Time  Line Contents
    49                                           def match_all(snap_ref, snap_exp, gns=[]):
    50                                               """ Try matching all halos in snap_ref with given group numbers with
    51                                               halos in snap_exp with the same set of group numbers. 
    52                                               
    53                                               Parameters
    54                                               ----------
    55                                               snap_ref : Snapshot object
    56                                                   Reference snapshot.
    57                                               snap_exp : Snapshot object
    58                                            

# Snapshot object 

Initialization time, when the data files have been generated

In [9]:
%lprun -f snap_ref.get_subhalos_IDs snap_ref.get_subhalos_IDs(1)

Timer unit: 1e-06 s

Total time: 0.47445 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/snapshot_obj.py
Function: get_subhalos_IDs at line 177

Line #      Hits         Time  Per Hit   % Time  Line Contents
   177                                               def get_subhalos_IDs(self, part_type, fnums=[]):
   178                                                   """ Read IDs of bound particles of given type for each halo.
   179                                                   
   180                                                   Paramaters
   181                                                   ----------
   182                                                   part_type : list of int
   183                                                       Types of particles, whose attribute values are retrieved (the
   184                                                       default is set for high-res part types)
   185                                                   fnums

In [ ]:
%lprun -f dataset_compute.compute_V1kpc dataset_compute.compute_V1kpc(snap_ref)

In [6]:
snap = snapshot_obj.Snapshot(sim_id,120)

In [7]:
%%timeit
snap = snapshot_obj.Snapshot(sim_id,120)

2.32 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Subhalo object

Analysis of halo tracing processing time:

In [30]:
halo = subhalo.Subhalo(sim_id, 127, 1, 0)

In [31]:
%lprun -f halo.trace halo.trace()

Timer unit: 1e-06 s

Total time: 28.1117 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/subhalo.py
Function: trace at line 61

Line #      Hits         Time  Per Hit   % Time  Line Contents
    61                                               def trace(self, stop=101):
    62                                                   """ Starting from the earliest identification, trace halo back
    63                                                   in time as far as possible.
    64                                           
    65                                                   Parameters
    66                                                   ----------
    67                                                   stop : int, optional
    68                                                       Earliest snapshot to be explored
    69                                           
    70                                                   Returns
    71                                  

In [21]:
%lprun -f trace_halo.find_match trace_halo.find_match(halo,127,snap)

Timer unit: 1e-06 s

Total time: 1.11174 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/trace_halo.py
Function: match_subhalo at line 364

Line #      Hits         Time  Per Hit   % Time  Line Contents
   364                                           def match_subhalo(subhalo, snap_id, snap_exp):
   365                                               """ Attempts to match a given subhalo with another subhalo in a given
   366                                               snapshot.
   367                                           
   368                                               Parameters
   369                                               ----------
   370                                               subhalo : Subhalo object
   371                                                   Matched subhalo.
   372                                               snap_id : int
   373                                                   ID of the snapshot, at which the match is searched

In [32]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%lprun -f halo.get_ids halo.get_ids(127)

Timer unit: 1e-06 s

Total time: 0.4894 s
File: /home/kasper/Curvaton_Simulations/APOSTLE_analysis/subhalo.py
Function: get_ids at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                               def get_ids(self, snap_id):
    45                                                   """ Retrieves the bound particle IDs in the given snapshot.
    46                                                   """
    47                                           
    48         1          2.0      2.0      0.0          gn, sgn = self.tracer[snap_id]
    49         1       4927.0   4927.0      1.0          snap = Snapshot(self.sim_id, snap_id)
    50         1      21959.0  21959.0      4.5          fnum = snap.file_of_halo(gn, sgn)
    51                                           
    52                                                   # Get index of halo:
    53         1      22576.0  22576.0      4.6          gns = snap.get_subhalos('Grou